In [1]:
import sys, os
current_ipynb_path = os.getcwd()

print(current_ipynb_path)
sys.path.insert(0, "../../")

import pandas as pd
import tensorflow as tf

from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.models.classifier.text_classifier import TextCNN

/home/web_server/antispam/project/zhouyulong/tf-library/demo




/home/web_server/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# 1. load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar_small/"
train_data = pd.read_csv(data_dir + 'train.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(train_data['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
# 2. create model
config = {
    "learning_rate": 0.01,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 256,
    "num_layers": 2,
    "conv_num_filters": 256,
    "keep_prob": 0.5,
    
    "gpu_id": 0,
    "gpu_num": 2,

    "model_output_dir": "./saved_models",
    "model_name": "cnn"
}

model = TextCNN(config, word_dict, name="default", distribute="tf_distribute")
model.build_model()
print(model.model_summary())




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
reduction_indices is deprecated, use axis instead



total parameter number: 3107586
                                        variable_name  variable_shape  \
0                   embedding_layer/embedding_table:0     [2634, 256]   
1                mw_conv_block/conv_1d/he_uniform/W:0   [3, 256, 256]   
2                           mw_conv_block/conv_1d/B:0           [256]   
3              mw_conv_block/conv_1d_1/he_uniform/W:0   [4, 256, 256]   
4                         mw_conv_block/conv_1d_1/B:0           [256]   
5              mw_conv_block/conv_1d_2/he_uniform/W:0   [5, 256, 256]   
6                         mw_conv_block/conv_1d_2/B:0           [256]   
7   transform2_block/multi_head_attention/conv_1d/...   [1, 256, 768]   
8   transform2_block/multi_head_attention/conv_1d/B:0           [768]   
9   transform2_block/multi_head_attention/conv_1d_...   [1, 25

In [4]:
model.train([train_data['text'], train_data['label']],
            batch_size=512,
            test_size=0.1,
            epochs=5,
            hooks=[])

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
build_train_graph: reconstructing variables...
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
build_train_graph (default): building inputs...
task: default, <class 'str'>
[[[<tf.Tensor 'data_batch:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_1:0' shape=(?,) dtype=string>, <tf.Tensor 'data_ba

  0%|          | 0/9 [00:00<?, ?it/s]                                                                 

'epoch 0 train time(sec): 8.488359212875366'
{'best_train_metric': 0.5360,
 'best_validate_metric': 0.5254,
 'epoch_index': 0,
 'train_loss': 0.7042,
 'train_metric': 0.5360,
 'validate_loss': 0.6887,
 'validate_metric': 0.5254}


task:default, lr:1.0000e-03, loss:0.6587, metric:0.6309:  22%|██▏       | 2/9 [00:00<00:00, 14.45it/s]

'epoch 1 train time(sec): 0.6256299018859863'
{'best_train_metric': 0.5618,
 'best_validate_metric': 0.6406,
 'epoch_index': 1,
 'train_loss': 0.6888,
 'train_metric': 0.5618,
 'validate_loss': 0.6566,
 'validate_metric': 0.6406}


task:default, lr:1.0000e-03, loss:0.6268, metric:0.6484:  22%|██▏       | 2/9 [00:00<00:00, 14.70it/s]

'epoch 2 train time(sec): 0.634943962097168'
{'best_train_metric': 0.6363,
 'best_validate_metric': 0.6895,
 'epoch_index': 2,
 'train_loss': 0.6397,
 'train_metric': 0.6363,
 'validate_loss': 0.5701,
 'validate_metric': 0.6895}


task:default, lr:1.0000e-03, loss:0.4571, metric:0.7930:  22%|██▏       | 2/9 [00:00<00:00, 14.39it/s]

'epoch 3 train time(sec): 0.6233561038970947'
{'best_train_metric': 0.7068,
 'best_validate_metric': 0.6973,
 'epoch_index': 3,
 'train_loss': 0.5613,
 'train_metric': 0.7068,
 'validate_loss': 0.5912,
 'validate_metric': 0.6973}


'epoch 4 train time(sec): 0.6362473964691162'
{'best_train_metric': 0.8056,
 'best_validate_metric': 0.7285,
 'epoch_index': 4,
 'train_loss': 0.4237,
 'train_metric': 0.8056,
 'validate_loss': 0.5833,
 'validate_metric': 0.7285}


In [5]:
# export model
model_path = "./saved_models/manually_saved_checkpoint.ckpt"
model.dump_checkpoint(model_path)
model.dump_model("./saved_models", "1", multi_replica_ckpt_path=model_path)
model.export_freeze_graph("./saved_models", "2", multi_replica_ckpt_path=model_path)

INFO:tensorflow:./saved_models/manually_saved_checkpoint.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Multi-replica model checkpoint saved in file: ./saved_models/manually_saved_checkpoint.ckpt

INFO:tensorflow:./saved_models/manually_saved_checkpoint.single_graph.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models/1/saved_model.pb
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 36 variables.
INFO:tensorflow:Converted 36 variables to const ops.
